In [7]:
import torch
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)
torch.manual_seed(2022)

# MovieLens数据集
- MovieLens数据集是电影网站提供的一份数据集，[原数据](https://grouplens.org/datasets/movielens/1m/)分为三个文件，users.dat movies.dat ratings.dat，包含了用户信息、电影信息和用户对电影的评分信息。

- 提供原始数据处理之后（参考examples/matching/data/ml-1m/preprocess_ml.py），全量数据集[**ml-1m.csv**](https://cowtransfer.com/s/5a3ab69ebd314e)

- 采样后的**ml-1m_sample.csv**(examples/matching/data/ml-1m/ml-1m_sample.csv)，是在全量数据中取出的前100个样本，调试用。在大家用ml-1m_sample.csv跑通代码后，便可以下载全量数据集测试效果，共100万个样本。

# 数据处理

In [6]:
import pandas as pd

data_path = "./mdata/"

unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
user = pd.read_csv(data_path+'ml-1m/users.dat',
                   sep='::',
                   header=None,
                   names=unames,
                   engine='python',
                   encoding="ISO-8859-1")
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',
                      sep='::',
                      header=None,
                      names=rnames,
                      engine='python',
                      encoding="ISO-8859-1")
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv(data_path+'ml-1m/movies.dat',
                     sep='::',
                     header=None,
                     names=mnames,
                     engine='python',
                     encoding="ISO-8859-1")

data = pd.merge(pd.merge(ratings, movies), user)
data.to_csv("./mdata/ml-1m.csv", index=False)

# 数据概述

|列名|用户ID<br>user_id|电影ID<br>movie_id|评级<br>rating|时间戳<br>timestamp|标题<br>title|流派<br>genres|性别<br>gender|年龄<br>age|职业<br>occupation|邮政编码<br>zip|
|:-|:-|:-|:-|:-|:-|:-|:-|:-|:-|:-|
|说明|1 到 6040|1 到 3952|5 星等级|以自 time(2) 返回的纪元以来的秒数表示||行动Action<br>冒险Adventure<br>动画片Animation<br>儿童Children's<br>喜剧Comedy<br>犯罪Comedy<br>纪录片Documentary<br>戏剧Drama<br>幻想Fantasy<br>黑色电影Film-Noir<br>恐怖Horror<br>音乐剧Musical<br>神秘Mystery<br>浪漫Romance<br>科幻Sci-Fi<br>惊悚片Thriller<br>战争War<br>西式WesternComedy|“M”表示男性<br>“F”表示女性|1：“18岁以下”<br>18：“18-24”<br>25：“25-34”<br>35：“35-44”<br>45：“45-49”<br>50：“50-55”<br>56：“56+”|0：“其他”或未指定"other" or not specified<br>1：“学术/教育家”"academic/educator"<br>2：“艺术家”"artist"<br>3：“文员/行政”"clerical/admin"<br>4：“大学生/研究生”"college/grad student"<br>5：“客户服务”"customer service"<br>6：“医生/保健”"doctor/health care"<br>7：“执行/管理”"executive/managerial"<br>8：“农民”"farmer"<br>9：“家庭主妇”"homemaker"<br>10：“K-12 学生”"K-12 student"<br>11：“律师”"lawyer"<br>12：“程序员”"programmer"<br>13：“退休”"retired"<br>14：“销售/营销”"sales/marketing"<br>15：“科学家”"scientist"<br>16：“个体户”"self-employed"<br>17：“技术员/工程师”"technician/engineer"<br>18：“商人/工匠”"tradesman/craftsman"<br>19：“失业”"unemployed"<br>20：“作家”"writer"||

文件包含约 3,900 部电影的 1,000,209 个匿名评级 由 2000 年加入 MovieLens 的 6,040 位 MovieLens 用户制作。
- 每个用户至少有20个评分
- 所有人口统计信息均由用户自愿提供，并且 未检查准确性。 仅提供一些人口统计数据的用户 信息包含在该数据集中。
- 标题与 IMDB 提供的标题相同（包括 发行年份）
- 由于意外重复，某些 MovieID 与电影不对应 条目和/或测试条目
- 电影大多是手工输入的，因此可能存在错误和不一致

In [19]:
file_path = './mdata/ml-1m.csv'
data = pd.read_csv(file_path)
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067


# DSSM模型

[DSSM 论文链接](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/cikm2013_DSSM_fullversion.pdf)

## 特征预处理
在本DSSM模型中，我们使用两种类别的特征，分别是稀疏特征（SparseFeature）和序列特征（SequenceFeature）。

- 对于稀疏特征，是一个离散的、有限的值（例如用户ID，一般会先进行LabelEncoding操作转化为连续整数值），模型将其输入到Embedding层，输出一个Embedding向量。

- 对于序列特征，每一个样本是一个List[SparseFeature]（一般是观看历史、搜索历史等），对于这种特征，默认对于每一个元素取Embedding后平均，输出一个Embedding向量。此外，除了平均，还有拼接，最值等方式，可以在pooling参数中指定。

- 框架还支持稠密特征（DenseFeature），即一个连续的特征值（例如概率），这种类型一般需归一化处理。但是本样例中未使用。

以上三类特征的定义在`torch_rechub/basic/features.py`

## 处理genres特征

In [20]:
# 处理genres特征，取出其第一个作为标签
data["cate_id"] = data["genres"].apply(lambda x: x.split("|")[0])
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip,cate_id
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067,Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067,Animation
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067,Musical
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067,Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067,Animation


## 稀疏特征->LabelEncoding

In [21]:
# 指定用户列和物品列的名字、离散和稠密特征，适配框架的接口
user_col, item_col = "user_id", "movie_id"
sparse_features = ['user_id', 'movie_id', 'gender', 'age', 'occupation', 'zip', "cate_id"]
dense_features = []

In [22]:
data[sparse_features].head()

,user_id,movie_id,gender,age,occupation,zip,cate_id
0,1,1193,F,1,10,48067,Drama
1,1,661,F,1,10,48067,Animation
2,1,914,F,1,10,48067,Musical
3,1,3408,F,1,10,48067,Drama
4,1,2355,F,1,10,48067,Animation


In [23]:
save_dir = './mdata/saved/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [24]:
# 对SparseFeature进行LabelEncoding
# 引入LabelEncoder库
from sklearn.preprocessing import LabelEncoder

# 创建一个字典来存储最大索引值
feature_max_idx = {}

# 遍历稀疏特征
for feature in sparse_features:
    # 实例化LabelEncoder对象
    lbe = LabelEncoder()
    
    # 将数据的当前特征编码并转换为整数，并加上1
    data[feature] = lbe.fit_transform(data[feature]) + 1
    
    # 获取当前特征的最大索引值，并加1后保存到字典中
    feature_max_idx[feature] = data[feature].max() + 1
    
    # 如果当前特征是用户列，创建一个映射字典：编码后的用户ID: 原始用户ID
    if feature == user_col:
        user_map = {encode_id + 1: raw_id for encode_id, raw_id in enumerate(lbe.classes_)}  #encode user id: raw user id
    
    # 如果当前特征是项目列，创建一个映射字典：编码后的项目ID: 原始项目ID
    if feature == item_col:
        item_map = {encode_id + 1: raw_id for encode_id, raw_id in enumerate(lbe.classes_)}  #encode item id: raw item id
        
# 将用户映射和项目映射保存到.npy文件中
np.save(save_dir+"raw_id_maps.npy", (user_map, item_map))  # evaluation时会用到
print('LabelEncoding后：')
data[sparse_features].head()

LabelEncoding后：


,user_id,movie_id,gender,age,occupation,zip,cate_id
0,1,1105,1,1,11,1589,8
1,1,640,1,1,11,1589,3
2,1,854,1,1,11,1589,12
3,1,3178,1,1,11,1589,8
4,1,2163,1,1,11,1589,3


In [25]:
dense_features

[]

## 用户塔与物品塔
在DSSM中，分为用户塔和物品塔，每一个塔的输出是用户/物品的特征拼接后经过MLP（多层感知机）得到的。
下面我们来定义物品塔和用户塔都有哪些特征：

In [26]:
# 定义两个塔对应哪些特征
user_cols = ["user_id", "gender", "age", "occupation", "zip"]
item_cols = ['movie_id', "cate_id"]

# 从data中取出相应的数据
user_profile = data[user_cols].drop_duplicates('user_id')
item_profile = data[item_cols].drop_duplicates('movie_id')
print(user_profile.head())
print(item_profile.head())

     user_id  gender  age  occupation   zip
0          1       1    1          11  1589
53         2       2    7          17  2249
182       12       2    3          13  1166
205       15       2    3           8   905
406       17       2    6           2  3188
   movie_id  cate_id
0      1105        8
1       640        3
2       854       12
3      3178        8
4      2163        3


In [27]:
from torch_rechub.utils.match import generate_seq_feature_match, gen_model_input
# 使用generate_seq_feature_match函数生成训练集和测试集
df_train, df_test = generate_seq_feature_match(data,
                                               user_col,
                                               item_col,
                                               time_col="timestamp",
                                               item_attribute_cols=[],
                                               sample_method=1,
                                               mode=0,
                                               neg_ratio=3,
                                               min_item=0)
print(df_train.head())

preprocess data


generate sequence features: 100%|█████████████████████████████████████████████████| 6040/6040 [00:08<00:00, 741.83it/s]


n_train: 3952516, n_test: 6040
0 cold start user droped 
   user_id  movie_id                                      hist_movie_id  histlen_movie_id  label
0     4994      3457  [2635, 41, 1132, 59, 1121, 2782, 3458, 2786, 2...                32      1
1     4131      1547  [2260, 347, 1902, 1237, 467, 2684, 2047, 2714,...               106      0
2     1334      1295                                             [2826]                 1      0
3     4414       892  [276, 3268, 1782, 1008, 2013, 580, 3551, 3376,...               122      0
4     3847      3050  [32, 1505, 254, 397, 1184, 48, 1789, 2652, 286...               214      0


在调用generate_seq_feature_match函数时，传入了以下几个参数：

data: 是一个包含原始数据的DataFrame对象。

user_col: 表示用户ID的列名。

item_col: 表示项目ID的列名。

time_col: 表示时间戳的列名，默认为"timestamp"。

item_attribute_cols: 是一个列表，表示项目属性的列名，这里为空列表。

sample_method: 表示采样方法，默认为1。

mode: 表示匹配模式，默认为0。

neg_ratio: 表示负样本的比例，默认为3。

min_item: 表示最小项目的数量，默认为0。

In [28]:
# 使用gen_model_input函数生成训练集和测试集的输入数据
x_train = gen_model_input(df_train, user_profile, user_col, item_profile, item_col, seq_max_len=50)
y_train = x_train["label"]
x_test = gen_model_input(df_test, user_profile, user_col, item_profile, item_col, seq_max_len=50)
y_test = x_test["label"]
# 打印出训练集输入数据前3个元素的字典表示
print({k: v[:3] for k, v in x_train.items()})

{'user_id': array([4994, 4131, 1334]), 'movie_id': array([3457, 1547, 1295]), 'hist_movie_id': array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0, 2635,   41, 1132,   59,
        1121, 2782, 3458, 2786, 2749, 2932, 2558, 2427, 2283, 2120, 1896,
        3435, 1874,  170, 3244, 1407, 1154, 3442,   47,   32, 3500, 3483,
        3489, 3518, 3529, 3466, 3492, 3456],
       [  29, 2375, 1004, 1026, 1960, 2666,  945,    2, 1836,  955,  309,
         358, 2938, 1920, 2427, 1981, 1788, 1826, 3247, 1842, 1669, 3257,
         633, 1963, 3207,  746, 1925, 1289, 1818, 1130, 2453, 1167, 3340,
        2661, 1187, 2443, 2447, 2174, 2463,  499,  754, 1051, 2622, 2481,
        2495, 1858, 2241, 1967, 1861, 2518],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
 

In [29]:
# 引入features模块中的SparseFeature和SequenceFeature类
from torch_rechub.basic.features import SparseFeature, SequenceFeature

# 定义用户特征类型
user_features = [
    # 对于每个用户特征，创建一个SparseFeature对象，并设置vocab_size为该特征的最大索引值+1，embed_dim为16
    SparseFeature(feature_name, vocab_size=feature_max_idx[feature_name], embed_dim=16) for feature_name in user_cols
]
# 添加一个SequenceFeature对象，表示历史电影ID特征，其vocab_size为movie_id的最大索引值+1，embed_dim为16，pooling方法为"mean"，并与其他特征共享嵌入层
user_features += [
    SequenceFeature("hist_movie_id",
                    vocab_size=feature_max_idx["movie_id"],
                    embed_dim=16,
                    pooling="mean",
                    shared_with="movie_id")
]

# 定义项目特征类型
item_features = [
    # 对于每个项目特征，创建一个SparseFeature对象，并设置vocab_size为该特征的最大索引值+1，embed_dim为16
    SparseFeature(feature_name, vocab_size=feature_max_idx[feature_name], embed_dim=16) for feature_name in item_cols
]

# 打印用户特征和项目特征列表
print(user_features)
print(item_features)

[<torch_rechub.basic.features.SparseFeature object at 0x00000206C6FF6B60>, <torch_rechub.basic.features.SparseFeature object at 0x00000206C6FF4280>, <torch_rechub.basic.features.SparseFeature object at 0x00000206C6FF5A50>, <torch_rechub.basic.features.SparseFeature object at 0x00000206C6FF6890>, <torch_rechub.basic.features.SparseFeature object at 0x00000206C6FF53C0>, <torch_rechub.basic.features.SequenceFeature object at 0x00000206C6FF4700>]
[<torch_rechub.basic.features.SparseFeature object at 0x00000206C6FF5BD0>, <torch_rechub.basic.features.SparseFeature object at 0x00000206C6FF6C20>]


In [30]:
# 将dataframe转为dict
from torch_rechub.utils.data import df_to_dict
all_item = df_to_dict(item_profile)
test_user = x_test
print({k: v[:3] for k, v in all_item.items()})
print({k: v[0] for k, v in test_user.items()})

{'movie_id': array([1105,  640,  854]), 'cate_id': array([ 8,  3, 12])}
{'user_id': 4229, 'movie_id': 2311, 'hist_movie_id': array([1067, 1134, 2652, 2401, 1174,    1, 2129, 3030, 1706, 2203, 1614,
       1744, 1619,  348,   21, 1450, 2167, 2111, 2163,  530,  361, 1705,
       3031,  347, 2502, 1365, 1594, 1442, 2495, 2942,  584,  453,  574,
        746, 1567, 1568, 1833,  330,  225,   69, 1823,  198, 1739, 1293,
       3020,  148,  778, 1553, 2239, 2143]), 'histlen_movie_id': 62, 'label': 1, 'gender': 2, 'age': 4, 'occupation': 18, 'zip': 3032, 'cate_id': 5}


## 训练模型

- 根据之前的x_train字典和y_train等数据生成训练用的Dataloader（train_dl）、测试用的Dataloader（test_dl, item_dl）。

- 定义一个双塔DSSM模型，`user_features`表示用户塔有哪些特征，`user_params`表示用户塔的MLP的各层维度和激活函数。（Note：在这个样例中激活函数的选取对最终结果影响很大，调试时不要修改激活函数的参数）
- 定义一个召回训练器MatchTrainer，进行模型的训练。

In [31]:
# 引入DSSM模型和MatchTrainer类
from torch_rechub.models.matching import DSSM
from torch_rechub.trainers import MatchTrainer

# 引入data模块中的MatchDataGenerator类
from torch_rechub.utils.data import MatchDataGenerator

# 根据之前处理的数据拿到Dataloader
# 使用x_train、y_train以及test_user、all_item生成训练集和测试集的Dataloader，batch_size为256
dg = MatchDataGenerator(x=x_train, y=y_train)
train_dl, test_dl, item_dl = dg.generate_dataloader(test_user, all_item, batch_size=256)

# 定义模型
# 使用用户特征和项目特征创建一个DSSM模型对象
model = DSSM(user_features,
             item_features,
             temperature=0.02,
             user_params={
                 "dims": [256, 128, 64],  # 用户侧的网络层数量和维度大小
                 "activation": 'prelu',  # 用户侧的激活函数，这里使用prelu
             },
             item_params={
                 "dims": [256, 128, 64],  # 项目侧的网络层数量和维度大小
                 "activation": 'prelu',  # 项目侧的激活函数，这里使用prelu
             })

# 模型训练器
# 创建一个MatchTrainer对象，传入模型对象以及其他参数
trainer = MatchTrainer(model,
                       mode=0,  # 同上面的mode，需保持一致
                       optimizer_params={
                           "lr": 1e-4,  # 学习率
                           "weight_decay": 1e-6  # 权重衰减
                       },
                       n_epoch=1,  # 训练轮数
                       device='cuda',  # 使用GPU进行训练
                       model_path=save_dir)  # 模型保存路径

# 开始训练
# 调用fit方法开始训练
trainer.fit(train_dl)

epoch: 0


train: 100%|████████████████████████████████████████████████████████| 15440/15440 [02:06<00:00, 121.96it/s, loss=0.551]


### Milvus向量化召回 评估
- 使用trainer获取测试集中每个user的embedding和数据集中所有物品的embedding集合
- 用annoy构建物品embedding索引，对每个用户向量进行ANN（Approximate Nearest Neighbors）召回K个物品
- 查看topk评估指标，一般看recall、precision、hit

In [32]:
'''
这段代码是使用MatchTrainer对象的inference_embedding方法来获取用户和项目的嵌入向量。

在DSSM模型中，用户和项目被分别映射到一个低维向量空间中，这个向量空间被称为嵌入空间。通过获取用户和项目的嵌入向量，可以进一步进行推荐系统的后续处理，如计算相似度、聚类等。

具体来说，对于用户侧，它会从test_dl数据加载器中读取一批用户ID，然后将这些用户ID输入到模型中，得到每个用户的嵌入向量；对于项目侧，它会从item_dl数据加载器中读取一批项目ID，然后将这些项目ID输入到模型中，得到每个项目的嵌入向量。

最后，返回的是一个二维数组，其中每一行表示一个用户或项目的嵌入向量。
'''
user_embedding = trainer.inference_embedding(model=model, mode="user", data_loader=test_dl, model_path=save_dir)
item_embedding = trainer.inference_embedding(model=model, mode="item", data_loader=item_dl, model_path=save_dir)

item inference: 100%|██████████████████████████████████████████████████████████████████| 15/15 [00:03<00:00,  4.32it/s]


In [40]:
from pymilvus import Collection,CollectionSchema,DataType,FieldSchema,connections,utility

connections.connect("default", host="localhost", port="19530")
dim = 64
has = utility.has_collection("rechub")
#print(f"Does collection rechub exist? {has}")
if has:
    utility.drop_collection("rechub")
# Create collection
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=dim),
]
schema = CollectionSchema(fields=fields)
conn = Collection("rechub", schema=schema)


In [41]:
if torch.is_tensor(item_embedding):
    item_embedding = item_embedding.cpu().numpy()
conn.release()
entities = [[i for i in range(len(item_embedding))], item_embedding]
conn.insert(entities)
conn.flush()

In [42]:
print(
    f"Number of entities in Milvus: {conn.num_entities}"
)  # check the num_entites

Number of entities in Milvus: 3706


In [43]:
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
conn.create_index("embeddings", index)

Status(code=0, message=)

In [38]:
# conn.load()

Number of entities in Milvus: 0


In [44]:
import collections

raw_id_maps = save_dir+"raw_id_maps.npy"
user_map, item_map = np.load(raw_id_maps, allow_pickle=True)
match_res = collections.defaultdict(dict)  # user id -> predicted item ids

In [45]:
def process_result(results):
    idx_list = []
    score_list = []
    for r in results:
        temp_idx_list = []
        temp_score_list = []
        for i in range(len(r)):
            temp_idx_list.append(r[i].id)
            temp_score_list.append(r[i].distance)
        idx_list.append(temp_idx_list)
        score_list.append(temp_score_list)
    return idx_list, score_list

In [46]:
def query(v, n):
    if torch.is_tensor(v):
        v = v.cpu().numpy().reshape(-1, dim)
    conn.load()
    search_params = {"metric_type": "L2", "params": {"nprobe": 16}}
    results = conn.search(v, "embeddings", search_params, n)
    return process_result(results)

In [47]:
topk = 10
for user_id, user_emb in zip(test_user[user_col], user_embedding):
    items_idx, items_scores = query(v=user_emb, n=topk)  #the index of topk match items
    match_res[user_map[user_id]] = np.vectorize(item_map.get)(all_item[item_col][items_idx])

In [49]:
match_res

defaultdict(dict,
            {4229: array([[   1,  588,  594, 2080, 3114, 2018,  595, 2081, 2355, 1282]],
                   dtype=int64),
             4197: array([[ 608,  527,  296, 1213,   34, 2997, 1265, 2716, 1704, 2762]],
                   dtype=int64),
             3786: array([[ 608,   34,  296, 1265, 2997, 1213, 2716, 1704,  527, 2396]],
                   dtype=int64),
             3829: array([[  34,  608, 2716, 1265, 2997,  296,  356, 1213, 1704, 2396]],
                   dtype=int64),
             5116: array([[  34,  608, 2716, 1265,  356, 2997,  296, 1704, 1213, 2396]],
                   dtype=int64),
             4998: array([[  34, 2716,  356, 1265,  608, 2997, 1704, 2396,  296, 1968]],
                   dtype=int64),
             76: array([[ 608,  527,  296, 1213,   34, 2997, 2762, 2858, 1265, 1704]],
                   dtype=int64),
             5948: array([[ 608,   34, 1265, 2716,  296, 2997, 1704, 1213, 2396,  356]],
                   dtype=int64),
        

In [50]:
items_idx

[[40, 171, 33, 365, 8, 540, 671, 50, 17, 116]]

In [51]:
items_scores

[[2.980100631713867,
  3.001267910003662,
  3.025826930999756,
  3.026522397994995,
  3.0373892784118652,
  3.042531728744507,
  3.0502285957336426,
  3.058762550354004,
  3.0615100860595703,
  3.069953680038452]]

In [48]:
user_col='user_id'
item_col='movie_id'

In [52]:
data = pd.DataFrame({user_col: test_user[user_col], item_col: test_user[item_col]})
data[user_col] = data[user_col].map(user_map)
data[item_col] = data[item_col].map(item_map)
user_pos_item = data.groupby(user_col).agg(list).reset_index()
ground_truth = dict(zip(user_pos_item[user_col], user_pos_item[item_col]))  # user id -> ground truth

In [58]:
type(ground_truth[1])

list

In [73]:
type(ground_truth)

dict

In [85]:
ground_truth

{1: [48],
 2: [1687],
 3: [2081],
 4: [2951],
 5: [288],
 6: [597],
 7: [3107],
 8: [3257],
 9: [2294],
 10: [2252],
 11: [3182],
 12: [3362],
 13: [2822],
 14: [2731],
 15: [3510],
 16: [2701],
 17: [1617],
 18: [1683],
 19: [1234],
 20: [1371],
 21: [2800],
 22: [910],
 23: [2643],
 24: [2657],
 25: [3798],
 26: [2683],
 27: [1262],
 28: [2132],
 29: [2550],
 30: [3949],
 31: [3450],
 32: [3286],
 33: [2124],
 34: [3257],
 35: [1162],
 36: [3639],
 37: [788],
 38: [296],
 39: [86],
 40: [2817],
 41: [1643],
 42: [2371],
 43: [2858],
 44: [2297],
 45: [355],
 46: [3280],
 47: [34],
 48: [3051],
 49: [2311],
 50: [3861],
 51: [1895],
 52: [2706],
 53: [581],
 54: [3422],
 55: [356],
 56: [514],
 57: [1258],
 58: [2807],
 59: [1244],
 60: [1589],
 61: [3908],
 62: [2939],
 63: [541],
 64: [3005],
 65: [953],
 66: [1961],
 67: [902],
 68: [2797],
 69: [476],
 70: [3793],
 71: [2959],
 72: [3105],
 73: [3596],
 74: [123],
 75: [2761],
 76: [3794],
 77: [2907],
 78: [368],
 79: [2126],
 80

In [95]:
b = {k: list(v[0]) for k, v in dict(match_res).items()}
b

{4229: [1, 588, 594, 2080, 3114, 2018, 595, 2081, 2355, 1282],
 4197: [608, 527, 296, 1213, 34, 2997, 1265, 2716, 1704, 2762],
 3786: [608, 34, 296, 1265, 2997, 1213, 2716, 1704, 527, 2396],
 3829: [34, 608, 2716, 1265, 2997, 296, 356, 1213, 1704, 2396],
 5116: [34, 608, 2716, 1265, 356, 2997, 296, 1704, 1213, 2396],
 4998: [34, 2716, 356, 1265, 608, 2997, 1704, 2396, 296, 1968],
 76: [608, 527, 296, 1213, 34, 2997, 2762, 2858, 1265, 1704],
 5948: [608, 34, 1265, 2716, 296, 2997, 1704, 1213, 2396, 356],
 3081: [1196, 110, 3578, 1200, 480, 1197, 1304, 1580, 1291, 1610],
 2836: [1196, 110, 3578, 608, 1200, 527, 1197, 480, 1304, 296],
 5421: [34, 608, 2716, 1265, 296, 2997, 1213, 1704, 356, 2396],
 4009: [1196, 110, 3578, 1200, 480, 1197, 1304, 1580, 1291, 1610],
 4196: [1196, 110, 3578, 1200, 527, 608, 1197, 480, 1304, 1213],
 1741: [1, 588, 594, 2080, 3114, 2018, 595, 2081, 2355, 1282],
 5005: [1, 588, 594, 2080, 3114, 2018, 595, 2081, 2355, 1282],
 1149: [1196, 110, 3578, 1200, 480, 11

In [96]:
# from torch_rechub.basic.metric import topk_metrics
out = topk_metrics(y_true=ground_truth, y_pred=b, topKs=[topk])
out

defaultdict(list,
            {'NDCG': ['NDCG@10: 0.0112'],
             'MRR': ['MRR@10: 0.0076'],
             'Recall': ['Recall@10: 0.0233'],
             'Hit': ['Hit@10: 0.0233'],
             'Precision': ['Precision@10: 0.0023']})

这段代码是用于评估匹配算法性能的函数。它首先导入了collections、numpy和pandas库，以及torch_rechub库中的Milvus类和topk_metrics函数。

然后定义了一个match_evaluation函数，接受用户嵌入向量、项目嵌入向量、测试集用户数据、所有项目的ID、用户ID列名、项目ID列名、原始ID映射文件路径和topk参数作为输入。

在函数内部，首先打印出“evaluate embedding matching on test data”信息，然后创建一个Milvus对象，并使用项目嵌入向量对其进行拟合。

接下来，通过for循环遍历测试集用户数据中的每个用户ID和对应的用户嵌入向量，对每个用户ID进行最近邻搜索，得到topk个匹配的项目ID，并将其存储在一个字典中。

接着，将原始ID映射文件加载到内存中，然后根据用户ID和项目ID映射表，将测试集用户数据转换为原始的用户ID和项目ID形式，并将其分组并转换为一个字典，其中键为用户ID，值为对应的项目ID列表（即真实结果）。

最后，调用topk_metrics函数计算评估指标，并返回结果。

In [70]:
"""The metric module, it is used to provide some metrics for recommenders.
Available function:
- auc_score: compute AUC
- gauc_score: compute GAUC
- log_loss: compute LogLoss
- topk_metrics: compute topk metrics contains 'ndcg', 'mrr', 'recall', 'hit'
Authors: Qida Dong, dongjidan@126.com
"""
from sklearn.metrics import roc_auc_score
import numpy as np
from collections import defaultdict


def auc_score(y_true, y_pred):

	return roc_auc_score(y_true, y_pred)


def get_user_pred(y_true, y_pred, users):
	"""divide the result into different group by user id

	Args:
		y_true (array): all true labels of the data
		y_pred (array): the predicted score
		users (array): user id 

	Return:
		user_pred (dict): {userid: values}, key is user id and value is the labels and scores of each user
	"""
	user_pred = {}
	for i, u in enumerate(users):
		if u not in user_pred:
			user_pred[u] = {'y_true': [y_true[i]], 'y_pred': [y_pred[i]]}
		else:
			user_pred[u]['y_true'].append(y_true[i])
			user_pred[u]['y_pred'].append(y_pred[i])

	return user_pred


def gauc_score(y_true, y_pred, users, weights=None):
	"""compute GAUC

	Args: 
		y_true (array): dim(N, ), all true labels of the data
		y_pred (array): dim(N, ), the predicted score
		users (array): dim(N, ), user id 
		weight (dict): {userid: weight_value}, it contains weights for each group. 
				if it is None, the weight is equal to the number
				of times the user is recommended
	Return:
		score: float, GAUC
	"""
	assert len(y_true) == len(y_pred) and len(y_true) == len(users)

	user_pred = get_user_pred(y_true, y_pred, users)
	score = 0
	num = 0
	for u in user_pred.keys():
		auc = auc_score(user_pred[u]['y_true'], user_pred[u]['y_pred'])
		if weights is None:
			user_weight = len(user_pred[u]['y_true'])
		else:
			user_weight = weights[u]
		auc *= user_weight
		num += user_weight
		score += auc
	return score / num



def ndcg_score(y_true, y_pred, topKs=None):
	if topKs is None:
		topKs = [5]
	result = topk_metrics(y_true, y_pred, topKs)
	return result['NDCG']
	


def hit_score(y_true, y_pred, topKs=None):
	if topKs is None:
		topKs = [5]
	result = topk_metrics(y_true, y_pred, topKs)
	return result['Hit']


def mrr_score(y_true, y_pred, topKs=None):
	if topKs is None:
		topKs = [5]
	result = topk_metrics(y_true, y_pred, topKs)
	return result['MRR']


def recall_score(y_true, y_pred, topKs=None):
	if topKs is None:
		topKs = [5]
	result = topk_metrics(y_true, y_pred, topKs)
	return result['Recall']


def precision_score(y_true, y_pred, topKs=None):
	if topKs is None:
		topKs = [5]
	result = topk_metrics(y_true, y_pred, topKs)
	return result['Precision']


def topk_metrics(y_true, y_pred, topKs=None):
	"""choice topk metrics and compute it
	the metrics contains 'ndcg', 'mrr', 'recall', 'precision' and 'hit'

	Args:
		y_true (dict): {userid, item_ids}, the key is user id and the value is the list that contains the items the user interacted
		y_pred (dict): {userid, item_ids}, the key is user id and the value is the list that contains the items recommended  
		topKs (list or tuple): if you want to get top5 and top10, topKs=(5, 10)

	Return:
		results (dict): {metric_name: metric_values}, it contains five metrics, 'ndcg', 'recall', 'mrr', 'hit', 'precision'

	"""
	if topKs is None:
		topKs = [5]
	assert len(y_true) == len(y_pred)

	if not isinstance(topKs, (tuple, list)):
		raise ValueError('topKs wrong, it should be tuple or list')
	
	pred_array = []
	true_array = []
	for u in y_true.keys():
		pred_array.append(y_pred[u])
		true_array.append(y_true[u])
	ndcg_result = []
	mrr_result = []
	hit_result = []
	precision_result = []
	recall_result = []
	for idx in range(len(topKs)):
		ndcgs = 0
		mrrs = 0
		hits = 0
		precisions = 0
		recalls = 0
		gts = 0
		for i in range(len(true_array)):
			if len(true_array[i]) != 0:
				mrr_tmp = 0
				mrr_flag = True
				hit_tmp = 0
				dcg_tmp = 0
				idcg_tmp = 0
				for j in range(topKs[idx]):
					if pred_array[i][j] in true_array[i]:
						hit_tmp += 1.
						if mrr_flag:
							mrr_flag = False
							mrr_tmp = 1. / (1 + j)
						dcg_tmp += 1. / (np.log2(j + 2))
					if j < len(true_array[i]):
						idcg_tmp += 1. / (np.log2(j + 2))
				gts += len(true_array[i])
				hits += hit_tmp
				mrrs += mrr_tmp
				recalls += hit_tmp / len(true_array[i])
				precisions += hit_tmp / topKs[idx]
				if idcg_tmp != 0:
					ndcgs += dcg_tmp / idcg_tmp
		hit_result.append(round(hits / gts, 4))
		mrr_result.append(round(mrrs / len(pred_array), 4))
		recall_result.append(round(recalls / len(pred_array), 4))
		precision_result.append(round(precisions / len(pred_array), 4))
		ndcg_result.append(round(ndcgs / len(pred_array), 4))

	results = defaultdict(list)
	for idx in range(len(topKs)):

		output = f'NDCG@{topKs[idx]}: {ndcg_result[idx]}'
		results['NDCG'].append(output)

		output = f'MRR@{topKs[idx]}: {mrr_result[idx]}'
		results['MRR'].append(output)

		output = f'Recall@{topKs[idx]}: {recall_result[idx]}'
		results['Recall'].append(output)

		output = f'Hit@{topKs[idx]}: {hit_result[idx]}'
		results['Hit'].append(output)

		output = f'Precision@{topKs[idx]}: {precision_result[idx]}'
		results['Precision'].append(output)
	return results

def log_loss(y_true, y_pred):
	score = y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)
	return -score.sum() / len(y_true)

def Coverage(y_pred, all_items, topKs=None):
	"""compute the coverage
	This method measures the diversity of the recommended items 
	and the ability to explore the long-tailed items
	Arg:
		y_pred (dict): {userid, item_ids}, the key is user id and the value is the list that contains the items recommended  
		all_items (set): all unique items
	Return:
		result (list[float]): the list of coverage scores
	"""
	if topKs is None:
		topKs = [5]
	result = []
	for k in topKs:
		rec_items = set([])
		for u in y_pred.keys():
			tmp_items = set(y_pred[u][:k])
			rec_items = rec_items | tmp_items
		score = len(rec_items) * 1. / len(all_items)
		score = round(score, 4)
		result.append(f'Coverage@{k}: {score}')
	return result


# print(Coverage({'0':[0,1,2],'1':[1,3,4]}, {0,1,2,3,4,5}, [2,3]))

# pred = np.array([  0.3, 0.2, 0.5, 0.9, 0.7, 0.31, 0.8, 0.1, 0.4, 0.6])
# label = np.array([   1,   0,   0,   1,   0,   0,    1,   0,   0,   1])
# users_id = np.array([ 2,   1,   0,   2,   1,   0,    0,   2,   1,   1])

# print('auc: ', auc_score(label, pred))
# print('gauc: ', gauc_score(label, pred, users_id))
# print('log_loss: ', log_loss(label, pred))

# for mt in ['ndcg', 'mrr', 'recall', 'hit','s']:
# 	tm = topk_metrics(y_true, y_pred, users_id, 3, metric_type=mt)
# 	print(f'{mt}: {tm}')
# y_pred = {'0': [0, 1], '1': [0, 1], '2': [2, 3]}
# y_true = {'0': [1, 2], '1': [0, 1, 2], '2': [2, 3]}
# out = topk_metrics(y_true, y_pred, topKs=(1,2))
# ndcgs = ndcg_score(y_true,y_pred, topKs=(1,2))
# print(out)
# print(ndcgs)

# ground_truth, match_res = np.load("C:\\Users\\dongj\\Desktop/res.npy", allow_pickle=True)
# print(len(ground_truth),len(match_res))
# out = topk_metrics(y_true=ground_truth, y_pred=match_res, topKs=[50])
# print(out)

if __name__ == "__main__":
	y_pred = {'0': [0, 1], '1': [0, 1], '2': [2, 3]}
	y_true = {'0': [1, 2], '1': [0, 1, 2], '2': [2, 3]}
	out = topk_metrics(y_true, y_pred, topKs=(1,2))
	print(out)

defaultdict(<class 'list'>, {'NDCG': ['NDCG@1: 0.6667', 'NDCG@2: 0.7956'], 'MRR': ['MRR@1: 0.6667', 'MRR@2: 0.8333'], 'Recall': ['Recall@1: 0.2778', 'Recall@2: 0.7222'], 'Hit': ['Hit@1: 0.2857', 'Hit@2: 0.7143'], 'Precision': ['Precision@1: 0.6667', 'Precision@2: 0.8333']})
